# Example: Impact Fund Screening
In this example we score potential land acquisitions in batch. The workflow shows how to combine land-cover metrics, B-Score calculation and occurrence density.

In [ ]:
from pathlib import Path
import pandas as pd
from verdesat.services.bscore import compute_bscores
from verdesat.biodiv.gbif_validator import OccurrenceService, plot_score_vs_density

In [ ]:
geojson_path = "examples/reforestation_plots.geojson"

In [ ]:
# Load AOIs from GeoJSON
projects_path = geojson_path
projects = AOI.from_geojson(projects_path, id_col='ID2')

In [ ]:
df_scores = compute_bscores(projects_path, year=2021, output='examples/bscore.csv')
df_scores['id'] = df_scores['id'].astype(int)

In [ ]:
occ = OccurrenceService()
densities = []
for aoi in projects:
    aoi_gdf = gpd.GeoDataFrame({'geometry': [aoi.geometry]}, crs='EPSG:4326')
    gdf = occ.fetch_occurrences(aoi.geometry)
    area_km2 = aoi_gdf.to_crs(epsg=6933).area.iloc[0] / 1e6
    densities.append(occ.occurrence_density_km2(gdf, area_km2))
plot_score_vs_density(df_scores['bscore'].tolist(), densities, 'examples/score_vs_density.png')

The scatter plot compares biodiversity score with citizen-science occurrence density for each polygon.